##### importing libraries

In [1]:
import pandas as pd


##### importing the exel file

In [26]:
df = pd.read_excel(r"Online%20Retail.xlsx")
df.head(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [27]:
df['CustomerID'].dropna(inplace=True)

In [37]:
sold_quantity = len(df[df['Quantity'] > 0])
return_quantity = len(df[df['Quantity'] < 0])
return_pcrt = (return_quantity / len(df['Quantity'])) * 100

print(f"total sales transactions: {sold_quantity}")
print(f"total return transactions: {return_quantity}")
print(f"prcentage of return transactions: {return_pcrt:.2f}%")

total sales transactions: 531285
total return transactions: 10624
prcentage of return transactions: 1.96%


In [31]:
sales_df = df[df['Quantity'] > 0].copy()
return_df = df[df['Quantity'] < 0].copy()

In [38]:
sales_df['Total_sales'] = sales_df['Quantity'] * sales_df['UnitPrice']
sales_df.head(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Total_sales
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


##### monthly sales trade (line chart)

In [40]:
monthly_sales = sales_df['Total_sales'].groupby(sales_df['InvoiceDate'].dt.to_period('M')).sum()
monthly_sales.reset_index()

,InvoiceDate,Total_sales
0,2010-12,823746.140
1,2011-01,691364.560
2,2011-02,523631.890
3,2011-03,717639.360
4,2011-04,537808.621
5,2011-05,770536.020
6,2011-06,761739.900
7,2011-07,719221.191
8,2011-08,737014.260
9,2011-09,1058590.172


##### Top 10 products by revenue (bar chart)

In [51]:
top_10_products = sales_df.groupby('Description')['Total_sales'].sum()
top_10_products.reset_index().nlargest(10, 'Total_sales')

,Description,Total_sales
1072,DOTCOM POSTAGE,206248.77
2867,REGENCY CAKESTAND 3 TIER,174484.74
2401,"PAPER CRAFT , LITTLE BIRDIE",168469.60
3860,WHITE HANGING HEART T-LIGHT HOLDER,106292.77
2427,PARTY BUNTING,99504.33
1827,JUMBO BAG RED RETROSPOT,94340.05
2065,MEDIUM CERAMIC TOP STORAGE JAR,81700.92
2205,Manual,78112.82
2706,POSTAGE,78101.88
2755,RABBIT NIGHT LIGHT,66964.99


##### Sales by country (top 5 countries)

In [ ]:
top_countries = sales_df['Country'].groupby(sales_df['Total_sales']).sum()
top_countries.reset_index().nlargest(5, 'Total_sales')

,Total_sales,Country
4907,168469.60,United Kingdom
4906,77183.60,United Kingdom
4905,38970.00,United Kingdom
4904,13541.33,United Kingdom
4903,11062.06,United Kingdom


##### Average order value over time

In [72]:
avg_order_over_time = (sales_df['InvoiceDate'].dt.to_period('d').count()) / sales_df['Quantity'].sum()
print(f"{avg_order_over_time:.3f}")

0.094


In [65]:
oo = sales_df['InvoiceDate'].dt.to_period('M').count()
ww = sales_df['Quantity'].sum()
print(f"{oo.dtype}")
print(f"{ww.dtype}")
print(f"{oo / ww:.2f}")

int64
int64
0.09
